# Construction d'un dataset à features numériques intéressantes

In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
d = pd.read_pickle("../Création du Dataset/datasetv3.pkl")
dataset = d.copy()

# Fonctions de conversion 

In [4]:
def min_max_moy(datasetOutput,datasetInterm,colname,teamnum):
    datasetOutput[colname+teamnum+"_MIN"] = datasetInterm[['TOP'+teamnum+'_'+colname,'JGL'+teamnum+'_'+colname,'MID'+teamnum+'_'+colname,'ADC'+teamnum+'_'+colname,'SUP'+teamnum+'_'+colname]].min(axis=1)
    datasetOutput[colname+teamnum+"_MAX"] = datasetInterm[['TOP'+teamnum+'_'+colname,'JGL'+teamnum+'_'+colname,'MID'+teamnum+'_'+colname,'ADC'+teamnum+'_'+colname,'SUP'+teamnum+'_'+colname]].max(axis=1)
    datasetOutput[colname+teamnum+"_MOY"] = datasetInterm[['TOP'+teamnum+'_'+colname,'JGL'+teamnum+'_'+colname,'MID'+teamnum+'_'+colname,'ADC'+teamnum+'_'+colname,'SUP'+teamnum+'_'+colname]].mean(axis=1)

### Conversion du Rank
Convertir le rang en nombre de LP au dessus de Iron IV 0 LP

In [5]:
def add_rank_numerical_value_columns(datasetInput,datasetOutput):
    listRankCols = ['TOP0_RANK','TOP1_RANK','JGL0_RANK','JGL1_RANK','MID0_RANK','MID1_RANK','ADC0_RANK','ADC1_RANK','SUP0_RANK','SUP1_RANK']
    rankOrder = ['IRON','BRONZE','SILVER','GOLD','PLATINUM','DIAMOND','MASTER','GRANDMASTER','CHALLENGER']
    divisionOrder = ['IV','III','II','I']
    datasetInterm = pd.DataFrame()
    
    for col in listRankCols:
        rank = pd.DataFrame(datasetInput[col].tolist(),datasetInput.index,columns=['RANK_','DIVISION','LP'])
        for i in range(len(rankOrder)):
            rank.loc[rank['RANK_']==rankOrder[i], 'RANK_'] = i

        for i in range(len(divisionOrder)):
            rank.loc[rank['DIVISION']==divisionOrder[i], 'DIVISION'] = i
        
        datasetInterm[col+'_'] = rank['RANK_']*400 + rank['DIVISION']*100 + rank['LP']
    
    min_max_moy(datasetOutput,datasetInterm,'RANK_','0')
    min_max_moy(datasetOutput,datasetInterm,'RANK_','1')
    

In [6]:
#Test
df = pd.DataFrame()
add_rank_numerical_value_columns(dataset,df)
df

,RANK_0_MIN,RANK_0_MAX,RANK_0_MOY,RANK_1_MIN,RANK_1_MAX,RANK_1_MOY
EUW1_6140228604,1275.0,1800.0,1543.2,1360.0,1620.0,1550.0
EUW1_6141546191,1271.0,1863.0,1708.8,1822.0,1946.0,1851.6
EUW1_6140416681,825.0,1600.0,1211.8,1122.0,1744.0,1380.2
EUW1_6139592219,1530.0,1751.0,1640.4,1663.0,1816.0,1754.4
EUW1_6045620377,1260.0,2030.0,1701.2,1600.0,1939.0,1731.8
...,...,...,...,...,...,...
EUW1_6135052656,1465.0,1674.0,1606.4,1390.0,1700.0,1586.4
EUW1_6141595905,1276.0,1738.0,1571.8,1238.0,2000.0,1502.4
EUW1_6137361607,694.0,1720.0,1170.2,452.0,1558.0,1052.6
EUW1_6088300233,1001.0,2231.0,1561.4,665.0,1714.0,1317.0


### Conversion des KDA/KDAG
Calcul du ratio (K+A)/D

In [7]:
def add_kda_ratio_columns(datasetInput,datasetOutput):
    listRankCols = ['TOP0_KDA','TOP1_KDA','JGL0_KDA','JGL1_KDA','MID0_KDA','MID1_KDA','ADC0_KDA','ADC1_KDA','SUP0_KDA','SUP1_KDA']
    datasetInterm = pd.DataFrame()
    
    for col in listRankCols:
        kda = pd.DataFrame(datasetInput[col].tolist(),datasetInput.index,columns=['K','D','A'])
        datasetInterm[col+'_log_'] = (kda['K'] + kda['A']) / (kda['D']+1)
        kdag = pd.DataFrame(datasetInput[col+'G'].tolist(),datasetInput.index,columns=['K','D','A'])
        datasetInterm[col+'G'+'_log_'] = (kdag['K'] + kdag['A']) / (kdag['D']+1)
    
    datasetInterm.applymap(lambda x:np.log(1+x))
    
    min_max_moy(datasetOutput,datasetInterm,'KDA_log_','0')
    min_max_moy(datasetOutput,datasetInterm,'KDA_log_','1')
    min_max_moy(datasetOutput,datasetInterm,'KDAG_log_','0')
    min_max_moy(datasetOutput,datasetInterm,'KDAG_log_','1')
    

In [8]:
#Test
df = pd.DataFrame()
add_kda_ratio_columns(dataset,df)
df

,KDA_log_0_MIN,KDA_log_0_MAX,KDA_log_0_MOY,KDA_log_1_MIN,KDA_log_1_MAX,KDA_log_1_MOY,KDAG_log_0_MIN,KDAG_log_0_MAX,KDAG_log_0_MOY,KDAG_log_1_MIN,KDAG_log_1_MAX,KDAG_log_1_MOY
EUW1_6140228604,0.0,2.750000,0.956061,0.0,4.642857,1.991005,1.456522,2.000000,1.743835,1.368421,3.700000,2.726277
EUW1_6141546191,0.0,5.666667,2.354762,0.0,2.545455,1.516498,1.552632,3.250000,2.407814,1.375000,2.333333,1.833788
EUW1_6140416681,0.0,0.000000,0.000000,0.0,1.600000,0.473846,1.631579,3.450000,2.465020,1.311111,2.724138,2.032696
EUW1_6139592219,0.0,1.275862,0.486751,1.0,7.333333,2.716760,1.256410,2.571429,1.615919,0.795455,3.093750,1.825642
EUW1_6045620377,0.0,5.000000,1.000000,0.0,0.000000,0.000000,1.054054,3.111111,2.045852,1.666667,3.105263,2.157062
...,...,...,...,...,...,...,...,...,...,...,...,...
EUW1_6135052656,0.0,6.200000,2.040000,0.0,2.000000,0.400000,1.419355,6.200000,2.768603,0.547619,1.966667,1.428716
EUW1_6141595905,0.0,3.060606,0.901010,0.0,1.750000,0.350000,2.181818,7.230769,3.402603,1.642857,3.166667,2.256667
EUW1_6137361607,0.0,4.500000,0.900000,0.0,1.375000,0.275000,1.361111,3.000000,2.465799,2.024390,3.666667,3.022082
EUW1_6088300233,0.0,3.500000,0.700000,0.0,0.000000,0.000000,1.392157,3.454545,2.188451,1.555556,2.392857,2.053812


### Conversion des booléens
Calcul d'un "score", qui représente le nombre de joueurs possédant la caractéristique

In [9]:
def add_bool_score_column(datasetInput,datasetOutput, boolvar):
    listCols = ['TOP0_','TOP1_','JGL0_','JGL1_','MID0_','MID1_','ADC0_','ADC1_','SUP0_','SUP1_']
    datasetInterm = datasetInput[[c+boolvar for c in listCols]]
    datasetInterm.applymap(int)
    datasetOutput[boolvar+'_SCORE_0'] = datasetInterm['TOP0_'+boolvar]
    datasetOutput[boolvar+'_SCORE_0'] = 0
    datasetOutput[boolvar+'_SCORE_1'] = 0
    for i in range(len(listCols)):
        if i%2==0:
            datasetOutput[boolvar+'_SCORE_0'] = datasetOutput[boolvar+'_SCORE_0'] + datasetInterm[listCols[i]+boolvar]
        else:
            datasetOutput[boolvar+'_SCORE_1'] = datasetOutput[boolvar+'_SCORE_1'] + datasetInterm[listCols[i]+boolvar]

In [10]:
#Test
df = pd.DataFrame()
add_bool_score_column(dataset,df,'FILL')
add_bool_score_column(dataset,df,'HOT')
df

,FILL_SCORE_0,FILL_SCORE_1,HOT_SCORE_0,HOT_SCORE_1
EUW1_6140228604,2,1,0,1
EUW1_6141546191,2,1,0,0
EUW1_6140416681,5,3,0,0
EUW1_6139592219,1,1,0,0
EUW1_6045620377,5,2,0,0
...,...,...,...,...
EUW1_6135052656,3,3,0,0
EUW1_6141595905,2,1,0,1
EUW1_6137361607,3,3,1,1
EUW1_6088300233,3,3,1,0


### 4. Ajout des données qui doivent être mises en échelle log

In [11]:
def add_log_numerical_column(datasetInput,datasetOutput,col):
    listCols = ['TOP0_','TOP1_','JGL0_','JGL1_','MID0_','MID1_','ADC0_','ADC1_','SUP0_','SUP1_']
    datasetInterm = datasetInput[[c+col for c in listCols]]
    
    datasetInterm.applymap(lambda x:np.log(1+x))
    
    min_max_moy(datasetOutput,datasetInterm,col,'0')
    min_max_moy(datasetOutput,datasetInterm,col,'1')

In [12]:
#Test
df = pd.DataFrame()
add_log_numerical_column(dataset,df,'LVL')
df

,LVL0_MIN,LVL0_MAX,LVL0_MOY,LVL1_MIN,LVL1_MAX,LVL1_MOY
EUW1_6140228604,120,481,260.4,37,715,286.6
EUW1_6141546191,159,672,421.8,137,491,366.4
EUW1_6140416681,313,703,452.4,195,473,299.0
EUW1_6139592219,42,406,124.8,67,418,232.4
EUW1_6045620377,160,687,454.8,68,705,421.6
...,...,...,...,...,...,...
EUW1_6135052656,34,635,251.6,44,465,298.2
EUW1_6141595905,71,689,346.0,271,549,434.4
EUW1_6137361607,36,707,348.8,154,354,215.0
EUW1_6088300233,53,316,196.8,139,510,298.4


### 5. Ajout des données qui n'ont pas besoin d'être transformées

In [13]:
def add_numerical_column(datasetInput,datasetOutput,col):
    min_max_moy(datasetOutput,datasetInput,col,'0')
    min_max_moy(datasetOutput,datasetInput,col,'1')

In [14]:
#Test
df = pd.DataFrame()
add_numerical_column(dataset,df,'VS')
df

,VS0_MIN,VS0_MAX,VS0_MOY,VS1_MIN,VS1_MAX,VS1_MOY
EUW1_6140228604,0.00,1.02,0.356,0.00,0.00,0.000
EUW1_6141546191,0.00,0.00,0.000,-0.14,1.26,0.460
EUW1_6140416681,-2.36,1.04,-0.852,-1.34,3.52,0.852
EUW1_6139592219,-1.32,1.58,0.028,-1.42,1.74,-0.028
EUW1_6045620377,-5.58,0.66,-1.148,-0.68,1.94,1.148
...,...,...,...,...,...,...
EUW1_6135052656,1.70,3.64,2.640,-5.52,-0.84,-2.640
EUW1_6141595905,-2.18,0.24,-0.592,-1.28,2.78,0.592
EUW1_6137361607,-5.96,1.78,-1.556,0.94,1.98,1.556
EUW1_6088300233,-1.54,2.10,0.592,-4.88,1.78,-0.592


# Construction du dataset

In [15]:
d = pd.read_pickle("../Création du Dataset/datasetv4_base.pkl")
dataset_v3 = d.copy()
dataset_v4 = pd.DataFrame()

In [16]:
d2 = pd.read_pickle("../Création du Dataset/datasetv4_fillmissing.pkl")
dataset_v3_full = d2.copy()

In [17]:
dataset_v3

,Y,TOP0_LVL,TOP0_TOTAL,TOP0_GWR,TOP0_WR,TOP0_NB,TOP0_HOT,TOP0_FILL,TOP0_RANK,TOP0_KDA,...,SUP1_FILL,SUP1_RANK,SUP1_KDA,SUP1_KDAG,SUP1_VS,SUP1_MAS,SUP1_WCH,SUP1_LCH,SUP1_TOTCH,SUP1_WRCH
EUW1_6140228604,False,481,237,0.506329,1.0,1,False,False,"[PLATINUM, II, 0]","[3.0, 3.0, 8.0]",...,False,"[GOLD, I, 100]","[0.0, 0.0, 0.0]","[1.0, 3.0, 13.8]",-0.6,127312,30.0,19.0,49.0,0.612245
EUW1_6141546191,False,323,27,0.407407,1.0,4,False,False,"[GOLD, IV, 71]","[7.0, 5.0, 5.75]",...,True,"[PLATINUM, II, 25]","[0.0, 0.0, 0.0]","[7.8, 8.6, 5.4]",1.2,4773,0.0,2.0,2.0,0.000000
EUW1_6140416681,False,342,22,0.500000,0.5,0,False,True,"[SILVER, I, 22]","[0.0, 0.0, 0.0]",...,True,"[GOLD, IV, 34]","[0.0, 0.0, 0.0]","[8.0, 5.8, 7.4]",-0.2,114124,0.0,1.0,1.0,0.000000
EUW1_6139592219,False,42,68,0.544118,0.5,2,False,False,"[GOLD, I, 30]","[6.0, 8.5, 5.0]",...,True,"[PLATINUM, II, 16]","[6.0, 2.0, 16.0]","[8.2, 5.4, 11.6]",-0.5,5675,3.0,0.0,3.0,1.000000
EUW1_6045620377,True,307,215,0.511628,0.5,0,False,True,"[PLATINUM, IV, 0]","[0.0, 0.0, 0.0]",...,False,"[PLATINUM, III, 30]","[0.0, 0.0, 0.0]","[1.6, 6.8, 11.4]",5.8,171930,5.0,8.0,13.0,0.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EUW1_6135052656,True,34,47,0.574468,0.0,1,False,False,"[GOLD, II, 65]","[12.0, 9.0, 13.0]",...,True,"[GOLD, I, 83]","[0.0, 0.0, 0.0]","[5.0, 6.8, 9.4]",-2.8,1233,1.0,2.0,3.0,0.333333
EUW1_6141595905,True,463,420,0.507143,0.5,0,False,True,"[PLATINUM, IV, 0]","[0.0, 0.0, 0.0]",...,False,"[DIAMOND, IV, 0]","[0.6, 8.6, 16.2]","[0.6, 8.6, 16.2]",2.7,413954,54.0,44.0,98.0,0.551020
EUW1_6137361607,False,36,38,0.421053,1.0,1,False,False,"[BRONZE, II, 94]","[8.0, 1.0, 1.0]",...,True,"[BRONZE, I, 83]","[0.0, 0.0, 0.0]","[8.0, 5.2, 12.2]",5.9,100570,7.0,3.0,10.0,0.700000
EUW1_6088300233,False,123,400,0.487500,0.5,0,False,True,"[SILVER, II, 1]","[0.0, 0.0, 0.0]",...,True,"[PLATINUM, III, 14]","[0.0, 0.0, 0.0]","[5.6, 4.6, 7.8]",6.2,130,NaN,NaN,NaN,NaN


In [18]:
# Level
add_log_numerical_column(dataset_v3,dataset_v4,'LVL')

# Rank
add_rank_numerical_value_columns(dataset_v3,dataset_v4)

# Nombre de parties de la saison
add_log_numerical_column(dataset_v3,dataset_v4,'TOTAL')

# Nombre de victoires avec le champion sur la saison
### Waiting for updated dataset ###

# Nombre de défaites avec le champion sur la saison
### Waiting for updated dataset ###

# Winrate sur les 5 dernières parties
add_numerical_column(dataset_v3,dataset_v4,'GWR')

# KDA moyen sur les 5 dernières parties + KDA moyen sur le champion sur les 5 dernières parties
add_kda_ratio_columns(dataset_v3,dataset_v4)

# Nombre de parties jouées avec le champion sur les 5 dernières parties
add_numerical_column(dataset_v3,dataset_v4,'NB')

# Winrate sur le champion sur les 5 dernières parties
add_numerical_column(dataset_v3,dataset_v4,'WR')

# Nombre de joueurs fill
add_bool_score_column(dataset_v3,dataset_v4,'FILL')

# Nombre de joueurs en winstreak
add_bool_score_column(dataset_v3,dataset_v4,'HOT')

# Masteries
add_log_numerical_column(dataset_v3,dataset_v4,'MAS')

# Matchups
add_numerical_column(dataset_v3,dataset_v4,'VS')

#Ajouts colonnes Paul
#Winrate sur le champion sur la saison
add_numerical_column(dataset_v3_full,dataset_v4,'WRCH')

#Victoires sur le champion sur la saison
add_numerical_column(dataset_v3_full,dataset_v4,'WCH')

#Défaites sur le champion sur la saison
add_numerical_column(dataset_v3_full,dataset_v4,'LCH')

#Parties jouées sur le champion sur la saison
add_numerical_column(dataset_v3_full,dataset_v4,'TOTCH')



In [19]:
dataset_v3 #should be unchanged

,Y,TOP0_LVL,TOP0_TOTAL,TOP0_GWR,TOP0_WR,TOP0_NB,TOP0_HOT,TOP0_FILL,TOP0_RANK,TOP0_KDA,...,SUP1_FILL,SUP1_RANK,SUP1_KDA,SUP1_KDAG,SUP1_VS,SUP1_MAS,SUP1_WCH,SUP1_LCH,SUP1_TOTCH,SUP1_WRCH
EUW1_6140228604,False,481,237,0.506329,1.0,1,False,False,"[PLATINUM, II, 0]","[3.0, 3.0, 8.0]",...,False,"[GOLD, I, 100]","[0.0, 0.0, 0.0]","[1.0, 3.0, 13.8]",-0.6,127312,30.0,19.0,49.0,0.612245
EUW1_6141546191,False,323,27,0.407407,1.0,4,False,False,"[GOLD, IV, 71]","[7.0, 5.0, 5.75]",...,True,"[PLATINUM, II, 25]","[0.0, 0.0, 0.0]","[7.8, 8.6, 5.4]",1.2,4773,0.0,2.0,2.0,0.000000
EUW1_6140416681,False,342,22,0.500000,0.5,0,False,True,"[SILVER, I, 22]","[0.0, 0.0, 0.0]",...,True,"[GOLD, IV, 34]","[0.0, 0.0, 0.0]","[8.0, 5.8, 7.4]",-0.2,114124,0.0,1.0,1.0,0.000000
EUW1_6139592219,False,42,68,0.544118,0.5,2,False,False,"[GOLD, I, 30]","[6.0, 8.5, 5.0]",...,True,"[PLATINUM, II, 16]","[6.0, 2.0, 16.0]","[8.2, 5.4, 11.6]",-0.5,5675,3.0,0.0,3.0,1.000000
EUW1_6045620377,True,307,215,0.511628,0.5,0,False,True,"[PLATINUM, IV, 0]","[0.0, 0.0, 0.0]",...,False,"[PLATINUM, III, 30]","[0.0, 0.0, 0.0]","[1.6, 6.8, 11.4]",5.8,171930,5.0,8.0,13.0,0.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EUW1_6135052656,True,34,47,0.574468,0.0,1,False,False,"[GOLD, II, 65]","[12.0, 9.0, 13.0]",...,True,"[GOLD, I, 83]","[0.0, 0.0, 0.0]","[5.0, 6.8, 9.4]",-2.8,1233,1.0,2.0,3.0,0.333333
EUW1_6141595905,True,463,420,0.507143,0.5,0,False,True,"[PLATINUM, IV, 0]","[0.0, 0.0, 0.0]",...,False,"[DIAMOND, IV, 0]","[0.6, 8.6, 16.2]","[0.6, 8.6, 16.2]",2.7,413954,54.0,44.0,98.0,0.551020
EUW1_6137361607,False,36,38,0.421053,1.0,1,False,False,"[BRONZE, II, 94]","[8.0, 1.0, 1.0]",...,True,"[BRONZE, I, 83]","[0.0, 0.0, 0.0]","[8.0, 5.2, 12.2]",5.9,100570,7.0,3.0,10.0,0.700000
EUW1_6088300233,False,123,400,0.487500,0.5,0,False,True,"[SILVER, II, 1]","[0.0, 0.0, 0.0]",...,True,"[PLATINUM, III, 14]","[0.0, 0.0, 0.0]","[5.6, 4.6, 7.8]",6.2,130,NaN,NaN,NaN,NaN


In [20]:
dataset_v4

,LVL0_MIN,LVL0_MAX,LVL0_MOY,LVL1_MIN,LVL1_MAX,LVL1_MOY,RANK_0_MIN,RANK_0_MAX,RANK_0_MOY,RANK_1_MIN,...,LCH0_MOY,LCH1_MIN,LCH1_MAX,LCH1_MOY,TOTCH0_MIN,TOTCH0_MAX,TOTCH0_MOY,TOTCH1_MIN,TOTCH1_MAX,TOTCH1_MOY
EUW1_6140228604,120,481,260.4,37,715,286.6,1275.0,1800.0,1543.2,1360.0,...,12.0,5.0,87.0,42.6,1.0,82.0,26.8,7.0,171.0,92.0
EUW1_6141546191,159,672,421.8,137,491,366.4,1271.0,1863.0,1708.8,1822.0,...,28.8,2.0,80.0,29.8,32.0,105.0,61.6,2.0,180.0,63.0
EUW1_6140416681,313,703,452.4,195,473,299.0,825.0,1600.0,1211.8,1122.0,...,6.4,1.0,90.0,31.8,0.0,24.0,13.6,1.0,186.0,65.0
EUW1_6139592219,42,406,124.8,67,418,232.4,1530.0,1751.0,1640.4,1663.0,...,35.0,0.0,203.0,54.0,7.0,256.0,82.4,3.0,409.0,108.8
EUW1_6045620377,160,687,454.8,68,705,421.6,1260.0,2030.0,1701.2,1600.0,...,103.2,4.0,26.0,10.4,0.0,558.0,220.4,7.0,63.0,21.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EUW1_6135052656,34,635,251.6,44,465,298.2,1465.0,1674.0,1606.4,1390.0,...,72.2,0.0,56.0,14.4,10.0,611.0,150.4,0.0,127.0,32.2
EUW1_6141595905,71,689,346.0,271,549,434.4,1276.0,1738.0,1571.8,1238.0,...,9.6,4.0,44.0,15.4,2.0,69.0,20.4,8.0,98.0,33.8
EUW1_6137361607,36,707,348.8,154,354,215.0,694.0,1720.0,1170.2,452.0,...,7.8,1.0,9.0,3.8,1.0,24.0,16.6,1.0,20.0,8.0
EUW1_6088300233,53,316,196.8,139,510,298.4,1001.0,2231.0,1561.4,665.0,...,8.8,1.0,9.0,5.0,1.0,37.0,16.8,1.0,20.0,10.4


In [21]:
dataset_v4.to_pickle("../Création du Dataset/datasetv4_transformed.pkl")
dataset_v4.to_csv("../Création du Dataset/datasetv4_transformed.csv")
